In [2]:
#Loading Dependencies 
import pandas as pd
import numpy as np
from numpy import loadtxt
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold

In [3]:
#Importing DATA
df = pd.read_csv("data_exercise_40.csv")
data1 = pd.read_csv("data_exercise_40.csv", usecols = range(1,120), header = 0)
data1 = data1.fillna(0)
s1 = pd.read_csv("data_exercise_40.csv", usecols = range(120,121) , header = 0)
s1 = s1.fillna(0)

s = '_24_Weekly_Exercise_Score'
uscore=sorted(list(set(s1[s])))
n_class=len(uscore)

In [4]:
#Split dataset
seed = 0
test_size = .2
X_train, X_test, y_train, y_test = train_test_split(data1, s1, test_size=test_size, random_state=seed)

In [5]:
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', missing = None, seed = 42)
clf_xgb.fit(X_train, 
            y_train.values.ravel(), 
            verbose = True, 
            early_stopping_rounds = 10, 
            eval_metric = 'aucpr', 
            eval_set = [(X_test, y_test)])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGBoostError: [15:49:52] C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\metric\rank_metric.cc:612: Check failed: preds.Size() == info.labels_.Size() (8784 vs. 488) : label size predict size not match

In [6]:
#convert data into data matrix
train = xgb.DMatrix(X_train, label = y_train)
test = xgb.DMatrix(X_test, label = y_test)

In [11]:
#set paramteres
param = {
    'max_depth': 4,
    'eta': .3,
    'objective': 'multi:softprob',
    'num_class': 18}
epochs = 10

In [16]:
model = xgb.train(param, train, epochs)
predictions = mode.predict(test)
print(predictions)
accuracy_score(y_test, predictions)

XGBoostError: [17:32:12] C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/multiclass_obj.cu:117: SoftmaxMultiClassObj: label must be in [0, num_class).

In [5]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10, verbosity = 0)

xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 26.247778


In [ ]:
data_dmatrix = xgb.DMatrix(data1,s1)

params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
cv_results.head()

In [ ]:
print((cv_results["test-rmse-mean"]).tail(1))

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()

In [ ]:
start_time=time.time()


model = XGBRegressor()
#model.fit(X_train, y_train.values.ravel())

param_grid = {
        'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'min_child_weight': np.arange(0.0001, 0.5, 0.001),
        'gamma': np.arange(0.0,40.0,0.005),
        'learning_rate': np.arange(0.0005,0.3,0.0005),
        'subsample': np.arange(0.01,1.0,0.01),
        'colsample_bylevel': np.round(np.arange(0.1,1.0,0.01)),
        'colsample_bytree': np.arange(0.1,1.0,0.01)}

kfold = KFold(n_splits=10, shuffle=True, random_state=10)
grid_search = RandomizedSearchCV(model, param_grid, scoring="accuracy", n_iter = 500, cv=kfold)
grid_result = grid_search.fit(data1, s1)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_[ 'mean_test_score' ]
stds = grid_result.cv_results_[ 'std_test_score' ]
params = grid_result.cv_results_[ 'params' ]

print(time.time()-start_time)